In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from PIL import Image
import face_recognition
import time
import imutils
import pickle

%matplotlib inline

In [11]:
#get paths of each file in folder named Images
#Images here contains my data(folders of various persons)
image_paths = glob('../data/calibration_images/*')
knownEncodings = []
knownNames = []
# loop over the image paths
for image_path in image_paths:
    if not 'jpg' in image_path.lower() and not 'png' in image_path.lower():
            print(f"Skipped {image_path}")
            continue
    name = 'potato'
    image = cv2.imread(image_path)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #Use Face_recognition to locate faces
    boxes = face_recognition.face_locations(rgb,model='hog')
    # compute the facial embedding for the face
    encodings = face_recognition.face_encodings(rgb, boxes)
    # loop over the encodings
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)
#save emcodings along with their names in dictionary data
data = {"encodings": knownEncodings, "names": knownNames}
#use pickle to save data into a file for later use
f = open("../data/face_enc", "wb")
f.write(pickle.dumps(data))
f.close()

Skipped ../data/calibration_images/processed_images


In [2]:
data = pickle.loads(open('../data/face_enc', "rb").read())

In [ ]:
def face_detect(frame):
    haar = cv2.CascadeClassifier('../data/haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = haar.detectMultiScale(gray,
                                 scaleFactor=1.1,
                                 minNeighbors=5,
                                 minSize=(60, 60),
                                 flags=cv2.CASCADE_SCALE_IMAGE)
    
    for x,y,w,h in faces:
        cv2.rectangle(image, (x,y), (x+w,y+h),(0, 255,255),10)
    
    return image

In [ ]:
print("Streaming started")
video_capture = cv2.VideoCapture(0)

frames = 0
while True:
    # grab the frame from the threaded video stream
    ret, frame = video_capture.read()
    if ret == False:
            print(f"Processed {frames} frames")
            break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faceCascade = cv2.CascadeClassifier('../data/haarcascade_frontalface_default.xml')
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
 
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    encodings = face_recognition.face_encodings(rgb)
    names = []
    # loop over the facial embeddings incase
    # we have multiple embeddings for multiple faces
    for encoding in encodings:
       #Compare encodings with encodings in data["encodings"]
       #Matches contain array with boolean values and True for the embeddings it matches closely
       #and False for rest
        matches = face_recognition.compare_faces(data["encodings"], encoding)
        #set name =inknown if no encoding matches
        name = "Not potato"
        # check to see if we have found a match
        if True in matches:
            #Find positions at which we get True and store them
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            # loop over the matched indexes and maintain a count for
            # each recognized face face
            for i in matchedIdxs:
                #Check the names at respective indexes we stored in matchedIdxs
                name = data["names"][i]
                #increase count for the name we got
                counts[name] = counts.get(name, 0) + 1
            #set name which has highest count
            name = max(counts, key=counts.get)
 
 
        # update the list of names
        names.append(name)
        # loop over the recognized faces
        for ((x, y, w, h), name) in zip(faces, names):
            # rescale the face coordinates
            # draw the predicted face name on the image
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
             0.75, (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break
    frames += 1
video_capture.release()
cv2.destroyAllWindows()

Streaming started


In [2]:
# load the harcaascade in the cascade classifier
faceCascade = cv2.CascadeClassifier('../data/haarcascade_frontalface_default.xml')
# load the known faces and embeddings saved in last file
data = pickle.loads(open('../data/face_enc', "rb").read())
#Find path to the image you want to detect face and pass it here
image = cv2.imread('../data/me.jpg')
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#convert image to Greyscale for haarcascade
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = faceCascade.detectMultiScale(gray,
                                     scaleFactor=1.1,
                                     minNeighbors=5,
                                     minSize=(60, 60),
                                     flags=cv2.CASCADE_SCALE_IMAGE)
 
# the facial embeddings for face in input
encodings = face_recognition.face_encodings(rgb)
names = []
# loop over the facial embeddings incase
# we have multiple embeddings for multiple fcaes
for encoding in encodings:
    matches = face_recognition.compare_faces(data["encodings"],encoding)
    #set name =inknown if no encoding matches
    name = "Unknown"
    # check to see if we have found a match
    if True in matches:
        #Find positions at which we get True and store them
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
            #Check the names at respective indexes we stored in matchedIdxs
            name = data["names"][i]
            #increase count for the name we got
            counts[name] = counts.get(name, 0) + 1
            #set name which has highest count
            name = max(counts, key=counts.get)
 
 
        # update the list of names
        names.append(name)
        # loop over the recognized faces
        for ((x, y, w, h), name) in zip(faces, names):
            # rescale the face coordinates
            # draw the predicted face name on the image
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
            cv2.imwrite('../data/me_labled.png',image)
    cv2.imshow("Frame", image)
    cv2.waitKey(0)